In [1]:
print('kaj so to pajki?')

kaj so to pajki?


In [2]:
# Simple usage of urllib to scrape a static website

import urllib

WEB_PAGE_ADDRESS = "https://vreme.arso.gov.si"

request = urllib.request.Request(
    WEB_PAGE_ADDRESS, 
    headers={'User-Agent': 'fri-ieps-TEST'}
)

with urllib.request.urlopen(request) as response: 
    html = response.read().decode("utf-8")
    print(html)

<!DOCTYPE html><html><head>
  <meta charset="utf-8">
  <title>ARSO vreme</title>
  <base href="/">

  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="icon" type="image/x-icon" href="favicon.ico">
  <style>html, 
body {
    height: 100%;
    margin:0;
}

.loading {
    background: #0082bc;
    color: #FFFFFF;
    width:100%;
    height:100%;
    margin:0;
}

.loading span {
    display:none;
    position:absolute;
    top:20px;
    left:20px;
    font-family: 'Open Sans', arial, helvetica, sans-serif;
}

.loading img {
    width:150px;
    height:22px;
    margin-left:100px;
    margin-top:50px;
}</style><link rel="stylesheet" type="text/css" href="default.css" media="print" onload="this.media='all'"><noscript><link rel="stylesheet" href="default.css"></noscript>
  <style>/* Slider */
</style><link rel="stylesheet" type="text/css" href="slick.css" media="print" onload="this.media='all'"><noscript><link rel="stylesheet" href="slick.css"></noscript>
  <st

In [10]:
# Connect to Flask API

import requests
from IPython.display import display, HTML

AUTH = ("admin", "admin") # Development authentication
ENDPOINT = "https://api:5000" # Accessible inside Docker network

response = requests.get(ENDPOINT + "/", verify = False, auth = AUTH)

# Check if the request was successful
if response.status_code == 200:
    # Get the HTML content from the response
    html_content = response.text
    # Display the HTML content
    display(HTML(html_content))
else:
    print("Failed to retrieve HTML content:", response.status_code)

/opt/conda/envs/wier/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [11]:
# Test database connection

response = requests.get(ENDPOINT + "/test_connection", verify = False, auth = AUTH).json()

if response['success'] == True:
    print(response['message'])
else:
    print(response['error'])

/opt/conda/envs/wier/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Connection to database successful!


In [16]:
# Execute an SQL script

# Enter the SQL script you want to execute
script = 'script_name.sql'

response = requests.post(ENDPOINT + f"/execute_script/{script}", verify = False, auth = AUTH).json()

if response['success'] == True:
    print(response['message'])
else:
    print(response['error'])

Script 'script_name.sql' does not exist.


/opt/conda/envs/wier/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [18]:
# Drop the whole database

response = requests.post(ENDPOINT + "/drop_db", verify = False, auth = AUTH).json()

if response['success'] == True:
    print(response['message'])
else:
    print(response['error'])

This operation is disabled! Enable it by commenting out the code.


/opt/conda/envs/wier/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [14]:
# Get all tables from the database

response = requests.get(ENDPOINT + "/get_tables", verify = False, auth = AUTH).json()

if response['success'] == True:
    print(response['message'])
    print(response['data'])
else:
    print(response['error'])

/opt/conda/envs/wier/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Successfully retrieved all tables from 'crawldb'.
['page', 'image', 'link', 'site', 'page_type', 'page_data', 'data_type']
